In [1]:
import os
import pandas as pd


PATH = "/home/haasehelen/haasehelen/ifwaste/analysis/waste_results/"
DATA_PATH = "/blue/carpena/haasehelen/ifwaste/output/multiple_stores/"
CURR_EXP = "run_1/"
#file_names = ['log_bought', 'log_eaten', 'log_wasted', 'log_hh_config', "log_still_have",
#                "log_hh_daily", "log_sim_config", "log_store_daily", "aggregated_outputs"]
file_names = ["log_wasted"]
#file_names = ["aggregated_outputs"]
data_dict = {}
current_path = os.path.join(DATA_PATH + CURR_EXP)
for file in file_names:
    file_path = os.path.join(current_path + file + ".csv")
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        df["run"] = 0
        if "day" in df.columns:
            df["day"] = df["day"].astype(int)
        data_dict[file] = df

In [2]:

food_categories = [
    'Meat & Fish', 'Dairy & Eggs', 'Fruits & Vegetables', 'Dry Foods', 'Baked Goods',
    'Snacks, Condiments, Liquids, Oils, Grease, & Other', 'Store-Prepared Items'
]

fg_names = ["FGMEAT","FGDAIRY","FGVEGETABLE","FGDRYFOOD","FGBAKED","FGSNACKS","FGSTOREPREPARED"]

lookup_df = pd.DataFrame({
    'Legend_name': food_categories,
    'Fg_name' : fg_names,
    'Servings_to_kg': [0.089, 0.146, 0.112, 0.076, 0.055, 0.071, 0.100],
    #"Color": ["#41afaa", "#00a0e1", "#466eb4", "#e6a532", "#d7642c","#af4b91"]
    "Color": ["#FFC446","#D66D10","#A40E4C","#2C2C54","#638EB5","#2F8550","#2C4A26"]
})

In [3]:
wasted = data_dict["log_wasted"]
wasted = wasted.astype({col: float for col in wasted.columns if col not in ["day", "household", "status", "reason"]})
wasted["household"] = wasted["household"].astype(int)
wasted["day"] = wasted["day"].astype(int)

for food in fg_names:
    conversion_factor = lookup_df.loc[lookup_df["Fg_name"] == food, "Servings_to_kg"].values[0]
    wasted.loc[:, food] = wasted[food] * conversion_factor

wasted["kg"] = wasted[fg_names].sum(axis=1)

In [4]:

sample_path = "/blue/carpena/haasehelen/ifwaste/input/gsua_based_configuration/samples/"
df_h = pd.read_csv( sample_path + "household_ifwaste_sample_df.txt", sep="\t") 
#df_nh = pd.read_csv( sample_path + "neighborhood_ifwaste_sample_df.txt", sep="\t") 
#df_sim = pd.read_csv( sample_path + "household_ifwaste_sample_df.txt", sep="\t")


In [5]:
df_h = df_h.reset_index().rename(columns={"index": "household"})
df_h = df_h[["household","Household:hh_amount_children","Household:hh_amount_adults"]]
df_h.rename(columns={"Household:hh_amount_children":"amount_children", "Household:hh_amount_adults":"amount_adults"}, inplace=True)

In [6]:
wasted_with_people = pd.merge(df_h, wasted, on='household', how='inner')  # 'inner' is default
wasted_with_people = wasted_with_people.drop(columns=["days_till_expiry", "run", "price"])
wasted_with_people = wasted_with_people[wasted_with_people["day"] > 13]

In [7]:
summed_weights = wasted_with_people.groupby('household')[fg_names + ["kg"]].sum().reset_index()
#wasted_results = pd.merge(wasted_with_people, summed_weights, on='household', how='inner')  # 'inner' is default
# Select only the relevant columns
people_per_hh = wasted_with_people[['household', 'amount_adults', 'amount_children']].drop_duplicates()
results = pd.merge(people_per_hh, summed_weights, on="household")

# 2. Count how often each family size appears
size_counts = (
    people_per_hh
    .groupby(['amount_adults', 'amount_children'])
    .size()
    .reset_index(name='count_households')
)
#size_counts = size_counts.drop(columns=["amount_adults", "amount_children"])

# 3. Merge the count back to your original household data
household_with_sizes = people_per_hh.merge(size_counts, on=['amount_adults', 'amount_children'], how='left')

# 5. Combine everything
final_df = household_with_sizes.merge(results, on='household', how='left', suffixes=("","_duplicate"))

final_df = final_df[[col for col in final_df.columns if not col.endswith('_duplicate')]]
final_df = final_df.drop(columns="household")
final_df

,amount_adults,amount_children,count_households,FGMEAT,FGDAIRY,FGVEGETABLE,FGDRYFOOD,FGBAKED,FGSNACKS,FGSTOREPREPARED,kg
0,2,0,276,15.181175,18.286792,31.503808,13.730464,14.689290,15.585849,44.4792,153.456578
1,2,0,276,14.456092,28.780542,7.205184,12.843392,3.251270,19.779890,54.1927,140.509070
2,2,5,6,69.691628,136.152884,98.184912,55.202980,56.328140,55.270873,112.6910,583.522417
3,2,5,6,25.081001,72.635876,34.063456,20.270872,13.439635,43.147197,83.1979,291.835937
4,1,0,12,6.753854,9.703890,5.036976,4.558252,2.704680,5.230073,29.9317,63.919425
...,...,...,...,...,...,...,...,...,...,...,...
1147,1,2,276,10.283327,27.191916,17.497760,9.050688,3.056515,12.899635,28.6596,108.639441
1148,1,2,276,16.657774,47.552054,10.721312,12.686984,1.852675,18.566500,33.8832,141.920499
1149,1,2,276,10.670032,22.026436,12.647264,7.801552,2.896685,9.863817,26.1368,92.042586
1150,2,0,276,9.792136,12.445332,7.655088,3.916432,3.657500,5.741841,24.2061,67.414429


In [ ]:
# Define aggregation dictionary
agg_dict = {col: 'mean' for col in final_df.columns if col not in ['amount_adults', 'amount_children', 'count_households']}
agg_dict['count_households'] = 'first'  # keep the first value

# Apply groupby and aggregation
final_df = final_df.groupby(['amount_adults', 'amount_children'], as_index=False).agg(agg_dict)

cols_to_scale = [col for col in final_df.columns if col not in ['amount_adults', 'amount_children', 'count_households']]
final_df[cols_to_scale] = final_df[cols_to_scale] / 100
final_df["kg_per_capita"] = final_df["kg"] / (final_df["amount_adults"] + final_df["amount_children"])
final_df

,amount_adults,amount_children,FGMEAT,FGDAIRY,FGVEGETABLE,FGDRYFOOD,FGBAKED,FGSNACKS,FGSTOREPREPARED,kg,count_households,kg_per_capita
0,1,0,0.072,0.053,0.075,0.028,0.034,0.039,0.206,0.507,12,0.507
1,1,2,0.140,0.276,0.169,0.096,0.057,0.115,0.251,1.104,276,0.368
2,1,3,0.216,0.372,0.270,0.141,0.140,0.222,0.452,1.813,6,0.453
3,1,4,0.234,0.296,0.348,0.082,0.115,0.108,0.218,1.401,270,0.280
4,1,5,0.357,0.481,0.469,0.150,0.134,0.170,0.475,2.237,6,0.373
5,1,6,0.536,0.767,0.650,0.232,0.284,0.289,0.736,3.494,6,0.499
6,2,0,0.155,0.255,0.200,0.092,0.089,0.124,0.430,1.345,276,0.672
7,2,2,0.257,0.462,0.330,0.159,0.163,0.267,0.717,2.355,12,0.589
8,2,3,0.371,0.450,0.495,0.132,0.207,0.187,0.722,2.565,6,0.513
9,2,4,0.271,0.347,0.397,0.077,0.136,0.095,0.235,1.558,6,0.260


In [9]:

stats_reason = wasted_with_people.pivot_table(
    index='household',
    columns='reason',
    values='kg',
    aggfunc='sum',
    fill_value=0
)


stats_reason.columns = [f"{col}" for col in stats_reason.columns]
column_names = list(stats_reason.columns)
stats_reason = pd.merge(people_per_hh, stats_reason, on='household', how='inner')  # 'inner' is default

summed_fg = stats_reason.groupby(by=['amount_adults', "amount_children"])[column_names].sum().reset_index()
stats_reason = stats_reason.drop(columns=["household"])

stats_reason = stats_reason.groupby(['amount_adults', 'amount_children'], as_index=False).mean()

cols_to_scale = [col for col in stats_reason.columns if col not in ['amount_adults', 'amount_children', 'count_households']]
stats_reason[cols_to_scale] = stats_reason[cols_to_scale] / 100
stats_reason

,amount_adults,amount_children,Inedible Parts,Plate Waste,Spoiled Food
0,1,0,0.102895,0.091882,0.312316
1,1,2,0.199752,0.817577,0.086543
2,1,3,0.298268,1.296165,0.218707
3,1,4,0.425373,0.842816,0.132733
4,1,5,0.500979,1.060681,0.674874
5,1,6,0.604970,1.653224,1.235418
6,2,0,0.235644,0.326100,0.782873
7,2,2,0.382251,0.832646,1.140450
8,2,3,0.458421,0.967792,1.139077
9,2,4,0.511726,0.928440,0.118246


In [10]:

stats_status = wasted_with_people.pivot_table(
    index='household',
    columns='status',
    values='kg',
    aggfunc='sum',
    fill_value=0
)


stats_status.columns = [f"{col}" for col in stats_status.columns]
column_names = list(stats_status.columns)
stats_status = pd.merge(people_per_hh, stats_status, on='household', how='inner')  # 'inner' is default

summed_fg = stats_status.groupby(by=['amount_adults', "amount_children"])[column_names].sum().reset_index()
stats_status = stats_status.drop(columns=["household"])

stats_status = stats_status.groupby(['amount_adults', 'amount_children'], as_index=False).mean()

cols_to_scale = [col for col in stats_status.columns if col not in ['amount_adults', 'amount_children', 'count_households']]
stats_status[cols_to_scale] = stats_status[cols_to_scale] / 100
stats_status

,amount_adults,amount_children,Prepared,Preprepared,Unprepared
0,1,0,0.163149,0.206172,0.137772
1,1,2,0.625481,0.251158,0.227231
2,1,3,1.052860,0.451991,0.308288
3,1,4,0.697085,0.218225,0.485613
4,1,5,0.941132,0.475385,0.820018
5,1,6,1.463732,0.736287,1.293593
6,2,0,0.594497,0.430344,0.319776
7,2,2,1.166981,0.717253,0.471113
8,2,3,0.872538,0.722139,0.970614
9,2,4,0.737011,0.235033,0.586368


In [13]:
merged = pd.merge(final_df, stats_reason, on=["amount_adults", "amount_children"], how='inner')  # 'inner' is default
merged = pd.merge(merged, stats_status, on=["amount_adults", "amount_children"], how='inner')  # 'inner' is default
merged = merged.round(decimals=3)
merged 


,amount_adults,amount_children,FGMEAT,FGDAIRY,FGVEGETABLE,FGDRYFOOD,FGBAKED,FGSNACKS,FGSTOREPREPARED,kg,count_households,kg_per_capita,Inedible Parts,Plate Waste,Spoiled Food,Prepared,Preprepared,Unprepared
0,1,0,0.072,0.053,0.075,0.028,0.034,0.039,0.206,0.507,12,0.507,0.103,0.092,0.312,0.163,0.206,0.138
1,1,2,0.140,0.276,0.169,0.096,0.057,0.115,0.251,1.104,276,0.368,0.200,0.818,0.087,0.625,0.251,0.227
2,1,3,0.216,0.372,0.270,0.141,0.140,0.222,0.452,1.813,6,0.453,0.298,1.296,0.219,1.053,0.452,0.308
3,1,4,0.234,0.296,0.348,0.082,0.115,0.108,0.218,1.401,270,0.280,0.425,0.843,0.133,0.697,0.218,0.486
4,1,5,0.357,0.481,0.469,0.150,0.134,0.170,0.475,2.237,6,0.373,0.501,1.061,0.675,0.941,0.475,0.820
5,1,6,0.536,0.767,0.650,0.232,0.284,0.289,0.736,3.494,6,0.499,0.605,1.653,1.235,1.464,0.736,1.294
6,2,0,0.155,0.255,0.200,0.092,0.089,0.124,0.430,1.345,276,0.672,0.236,0.326,0.783,0.594,0.430,0.320
7,2,2,0.257,0.462,0.330,0.159,0.163,0.267,0.717,2.355,12,0.589,0.382,0.833,1.140,1.167,0.717,0.471
8,2,3,0.371,0.450,0.495,0.132,0.207,0.187,0.722,2.565,6,0.513,0.458,0.968,1.139,0.873,0.722,0.971
9,2,4,0.271,0.347,0.397,0.077,0.136,0.095,0.235,1.558,6,0.260,0.512,0.928,0.118,0.737,0.235,0.586


In [15]:
# Select the columns you want to include
cols_to_export = ['amount_adults', 'amount_children', 'kg', 'kg_per_capita']  # update as needed

# Subset the DataFrame
latex_df = merged[cols_to_export]
latex_df = latex_df[latex_df["amount_adults"] > 1]

# Export to LaTeX
latex_code = latex_df.to_latex(index=False, caption="Food waste per household type", label="tab:fw_summary")

# Save to file (optional)
with open("fw_summary.tex", "w") as f:
    f.write(latex_code)

# Print the LaTeX string (optional)
print(latex_code)


\begin{table}
\caption{Food waste per household type}
\label{tab:fw_summary}
\begin{tabular}{rrrr}
\toprule
amount_adults & amount_children & kg & kg_per_capita \\
\midrule
2 & 0 & 1.345000 & 0.672000 \\
2 & 2 & 2.355000 & 0.589000 \\
2 & 3 & 2.565000 & 0.513000 \\
2 & 4 & 1.558000 & 0.260000 \\
2 & 5 & 3.939000 & 0.563000 \\
2 & 6 & 3.896000 & 0.487000 \\
\bottomrule
\end{tabular}
\end{table}

